In [1]:
#Your final output should provide a visualized summary of the sentiments expressed in Tweets sent 
#out by the following news organizations: BBC, CBS, CNN, Fox, and New York times.

In [3]:
#import dependencies
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Twitter API Keys
from config import (consumer_key, consumer_secret, 
                    access_token, access_token_secret)

In [2]:
#The first plot will be and/or feature the following:

#Be a scatter plot of sentiments of the last 100 tweets sent out by each news organization,
#ranging from -1.0 to 1.0, where a score of 0 expresses a neutral sentiment, -1 the most negative 
#sentiment possible, and +1 the most positive sentiment possible.

#Each plot point will reflect the compound sentiment of a tweet.

#Sort each plot point by its relative timestamp.

In [ ]:
#The second plot will be a bar plot visualizing the overall sentiments of the last 100 tweets from each 
#organization. For this plot, you will again aggregate the compound sentiments analyzed by VADER.